# Import libraries

In [ ]:
import os
import re
import pandas as pd

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store cleaned data

In [ ]:
os.makedirs('/content/drive/MyDrive/datasets/cleaning', exist_ok=True)

# Import datasets

## Get lists of files in **/content/drive/MyDrive/datasets/AWA_q1d_gpt-3.5-turbo-0301** and **/content/drive/MyDrive/datasets/AWA_q1d_gpt-3.5-turbo-0613** directories

In [ ]:
# List all files in directories. via
# https://www.geeksforgeeks.org/python-list-files-in-a-directory/
dir_0301 = '/content/drive/MyDrive/datasets/AWA_q1d_gpt-3.5-turbo-0301'
file_list_0301 = os.listdir(dir_0301)
file_list_0301 = sorted(file_list_0301)
dir_0613 = '/content/drive/MyDrive/datasets/AWA_q1d_gpt-3.5-turbo-0613'
file_list_0613 = os.listdir(dir_0613)
file_list_0613 = sorted(file_list_0613)

## Concatenate files in file lists into one Pandas DataFrame for each list

In [ ]:
# Import and concatenate multiple files. via
# https://stackoverflow.com/a/21232849
AWA_q1d_0301 = pd.concat(
    [pd.read_csv("".join((dir_0301, '/', file)), sep='\t') for file in
     file_list_0301],
    axis=1)
AWA_q1d_0613 = pd.concat(
    [pd.read_csv("".join((dir_0613, '/', file)), sep='\t') for file in
     file_list_0613],
    axis=1)
# Drop duplicate columns. via
# https://www.geeksforgeeks.org/
# how-to-find-drop-duplicate-columns-in-a-pandas-dataframe/
AWA_q1d_0301 = AWA_q1d_0301.T.drop_duplicates().T
AWA_q1d_0613 = AWA_q1d_0613.T.drop_duplicates().T

# Define a function to get the distribution of responses

In [ ]:
def get_responses_distribution(df, model):
    # Value counts across multiple columns. via
    # https://stackoverflow.com/a/61565732
    responses_distribution = df.iloc[:, 1:].stack().value_counts()
    # Convert value_counts output to dataframe format. via
    # https://stackoverflow.com/a/47136484
    responses_distribution = responses_distribution.rename_axis(
        'Responses').reset_index(name='Count')
    responses_distribution.to_csv(
        f'/content/drive/MyDrive/datasets/cleaning/'
        f'AWA_q1d_responses_distribution_{model}.csv',
        index=False)
    # Count null values in dataframe. via
    # https://stackoverflow.com/questions/26266362/
    # how-do-i-count-the-nan-values-in-a-column-in-pandas-dataframe#
    # comment74712638_26266451
    print('Number of unusable responses is:')
    print(df.iloc[:, 1:].isna().sum().sum())

In [ ]:
get_responses_distribution(AWA_q1d_0301, 'gpt-3.5-turbo-0301')
get_responses_distribution(AWA_q1d_0613, 'gpt-3.5-turbo-0613')

# Remove unsuccessful persona priming

In [ ]:
def rm_failed(x):
    x = str(x)
    x = x.lower()
    if 'as an ai language model' in x or 'as a language model' in x:
        return None
    else:
        return x

cols = list(range(21))
cols.remove(0) # tweet
cols.remove(14) # predicted_q1d_labels_base
AWA_q1d_0301.iloc[:, cols] = AWA_q1d_0301.iloc[:, cols].applymap(
    lambda x: rm_failed(x))
AWA_q1d_0613.iloc[:, cols] = AWA_q1d_0613.iloc[:, cols].applymap(
    lambda x: rm_failed(x))

In [ ]:
get_responses_distribution(AWA_q1d_0301, 'gpt-3.5-turbo-0301')
get_responses_distribution(AWA_q1d_0613, 'gpt-3.5-turbo-0613')

# Clean valid responses to conform to how the data was coded, e.g., `5` if `Answer: 5`

In [ ]:
def remap(x):
    # Determine if string only contains one digit. via
    # https://stackoverflow.com/a/39799113
    # Return True/False when using Regex. via
    # https://stackoverflow.com/a/6577018
    if bool(re.search("^\D*1\D*$", str(x))) or bool(
            re.search("^\D*1.0\D*$", str(x))):
        return str(1)
    elif bool(re.search("^\D*2\D*$", str(x))) or bool(
            re.search("^\D*2.0\D*$", str(x))):
        return str(2)
    elif bool(re.search("^\D*3\D*$", str(x))) or bool(
            re.search("^\D*3.0\D*$", str(x))):
        return str(3)
    elif bool(re.search("^\D*4\D*$", str(x))) or bool(
            re.search("^\D*4.0\D*$", str(x))):
        return str(4)
    elif bool(re.search("^\D*5\D*$", str(x))) or bool(
            re.search("^\D*5.0\D*$", str(x))):
        return str(5)
    else:
        return x


AWA_q1d_0301.iloc[:, 1:] = AWA_q1d_0301.iloc[:, 1:].applymap(
    lambda x: remap(x))
AWA_q1d_0613.iloc[:, 1:] = AWA_q1d_0613.iloc[:, 1:].applymap(
    lambda x: remap(x))

In [ ]:
get_responses_distribution(AWA_q1d_0301, 'gpt-3.5-turbo-0301')
get_responses_distribution(AWA_q1d_0613, 'gpt-3.5-turbo-0613')

# Clean remaining valid responses to conform to how the data was coded, e.g., `5` if `As a Native Hawaiian/Pacific Islander in the United States, I find this post extremely offensive and disrespectful. It perpetuates harmful racial stereotypes and uses derogatory language. On a scale of 1 to 5, I would rate this post as a 5, very much racist.` (not remapped in previous round due to digits other than `5` present) AND remove invalid responses

In [ ]:
def remap_again(x):
    if x is not None and ('a 5' in x or 'a "5"' in x):
        return str(5)
    elif x in {str(1), str(2), str(3), str(4), str(5)}:
        return x
    else:
        return None


AWA_q1d_0301.iloc[:, 1:] = AWA_q1d_0301.iloc[:, 1:].applymap(
    lambda x: remap_again(x))
AWA_q1d_0613.iloc[:, 1:] = AWA_q1d_0613.iloc[:, 1:].applymap(
    lambda x: remap_again(x))

In [ ]:
get_responses_distribution(AWA_q1d_0301, 'gpt-3.5-turbo-0301')
get_responses_distribution(AWA_q1d_0613, 'gpt-3.5-turbo-0613')

# Listwise deletion of cases with invalid responses

In [ ]:
# Select rows with null values. via
# https://stackoverflow.com/a/14247708
todrop = AWA_q1d_0301[AWA_q1d_0301[AWA_q1d_0301.columns[1:]].isna().any(axis=1)]['tweet'].tolist() + \
    AWA_q1d_0613[AWA_q1d_0613[AWA_q1d_0613.columns[1:]].isna().any(axis=1)]['tweet'].tolist()
# Drop rows where column values match any list element. via
# https://stackoverflow.com/a/27965417
AWA_q1d_0301 = AWA_q1d_0301[~AWA_q1d_0301['tweet'].isin(todrop)]
AWA_q1d_0301.reset_index(drop=True, inplace=True)
AWA_q1d_0613 = AWA_q1d_0613[~AWA_q1d_0613['tweet'].isin(todrop)]
AWA_q1d_0613.reset_index(drop=True, inplace=True)

# Save cleaned responses

In [ ]:
AWA_q1d_0301.to_csv(
    '/content/drive/MyDrive/datasets/cleaning/'
    'AWA_q1d_predictions_cleaned_gpt-3.5-turbo-0301.tsv',
    sep='\t', index=False)
AWA_q1d_0613.to_csv(
    '/content/drive/MyDrive/datasets/cleaning/'
    'AWA_q1d_predictions_cleaned_gpt-3.5-turbo-0613.tsv',
    sep='\t', index=False)